## Volatility Targeting backtesting using zipline

* import zipline and other required libraries

In [ ]:
import os
import time
import threading
import numpy as np
import pandas as pd
from openbb_terminal.sdk import openbb
import riskfolio as rp

from zipline import run_algorithm
from zipline.api import (
    symbol,
    date_rules,
    time_rules,
    get_datetime,
    schedule_function,
    calendars,
    get_open_orders,
    order_target_percent,
    set_commission,
    set_slippage,
    set_benchmark
)
from zipline.finance import commission, slippage
from zipline.data import bundles
from zipline.utils.run_algo import load_extensions
from zipline.errors import SymbolNotFound

import pyfolio as pf

* Setup Quandl to get data

In [ ]:
from dotenv  import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
NASDAQ_API_KEY = os.getenv("NASDAQ_KEY")

In [ ]:
os.environ["QUANDL_API_KEY"] = NASDAQ_API_KEY
bundle = "quandl"
bundles.ingest(bundle)

### Trade Settings

In [ ]:
bar_count = 66 # 22 trading days a month * 3
method_mu = "hist"
method_cov = "hist"
lower_ret = 0.0008 # risk free return: 0.08 bps

#### Stock Screening

In [ ]:
new_highs = openbb.stocks.screener.screener_data("new_high")
port_data = new_highs[
    (new_highs.Price > 15) &
    (new_highs.Country == "USA")
]

### Initiailize backtest

In [ ]:
def initialize(context):
    # from portdata above, get the list of tickers and check if ticker exists in ingested qandl data
    tickers = port_data.Ticker.tolist()
    
    context.assets = []
    for ticker in tickers:
        try:
            context.assets.append(symbol(ticker))
        except SymbolNotFound:
            print(f"{ticker} not found in {bundle} bundle. Skipping...")
    
    schedule_function(
        rebalance,
        date_rules.week_start(),
        time_rules.market_open(),
        calendar=calendars.US_EQUITIES,
    )

    # Set up the commission model to charge us per share and a volume slippage model
    set_commission(
        us_equities=commission.PerShare(
            cost=0.005,
            min_trade_cost=2.0
        )
    )
    set_slippage(
        us_equities=slippage.VolumeShareSlippage(
            volume_limit=0.0025, 
            price_impact=0.01
        )
    )
    # set_benchmark(symbol("SPY")) # free data don't have SPY, so manually download and pass it to zipline

In [ ]:
start = pd.Timestamp("2016-01-01")
end = pd.Timestamp("2017-12-31")

import pandas_datareader.data as web
sp500 = web.DataReader('SP500', 'fred', start, end).SP500
benchmark_returns = sp500.pct_change()

### Execute trades

 ##### We loop thru every ticker and if its tradeable and there are no open orders, then we order target percent and zipline rebalances

In [ ]:
def exec_trades(data, assets, weights):
    # Loop through every asset...
    for asset in assets:
        # ...if the asset is tradeable and there are no open orders...
        print(asset)
        if data.can_trade(asset) and asset in weights.index and not get_open_orders(asset):
            # ...execute the order against the target percent
            target_percent = weights.at[asset, "weights"]
            order_target_percent(asset, target_percent)

In [ ]:
def rebalance(context, data):
    
    assets = context.assets
    
    prices = data.history(
        assets,
        "price",
        bar_count=bar_count,
        frequency="1d"
    )
    
    returns = prices.pct_change()[1:]
    returns.dropna(how="any", axis=1, inplace=True)
    returns = returns.loc[:, (returns != 0).any(axis=0)]
    returns = returns.loc[:, np.isfinite(returns).all(axis=0)]
    
    port = rp.Portfolio(returns=returns)
    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)
    port.lowerret = lower_ret
    
    # try:
    weights = port.rp_optimization(
        model="Classic",
        rm="MV",
        hist=True,
        rf=0.05,
        b=None
    )
    # except:
    #     print(prices)

    print(
        f"{get_datetime().date()} {context.portfolio.portfolio_value}"
    )

    exec_trades(data, assets=assets, weights=weights)

In [ ]:
def analyze(context, perf):
    perf.portfolio_value.plot()

### Run the backtest

In [ ]:
perf = run_algorithm(
    start=pd.Timestamp("2016-01-01"),
    end=pd.Timestamp("2017-12-31"),
    initialize=initialize,
    analyze=analyze,
    capital_base=100_000,
    bundle=bundle,
    benchmark_returns=benchmark_returns,
)

In [ ]:
perf.alpha.plot()

In [ ]:
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)

In [ ]:
pf.create_full_tear_sheet(returns, positions, transactions)